In [17]:
import pandas as pd
import numpy as np
import spacy
from nltk.corpus import movie_reviews
from nltk.tokenize import sent_tokenize
from nltk.wsd import lesk

# Esercizio 1.4 - Teoria di Hanks

La Teoria di Hanks propone di guardare al verbo ed alla sua arità per trovare il significato di una frase.
Prevede un processo composto di tre passi:
1. estrazione dei _fillers_ del verbo
2. recupero dei _semantic types_ a partire dai _fillers_
3. clusterizzazione dei _semantic types_

Per questo esercizio è stato utilizzato il corpus _movie reviews_ di NLTK, contenente recensioni di film.

In [18]:
# import nltk
# nltk.download("movie_reviews")

In [19]:
# Scegliamo il verbo su cui operare
VERB = 'get'

## Estrazione dei fillers del verbo

Dal corpus recuperiamo 1000 frasi contenenti il verbo _get_.

In [20]:
def get_sentences(verb, corpus, num):
    """
    Given a corpus and a verb return the number of senteneces specified in the limit.

    :param verb: the verb as a string
    :param corpus: the corpus in which the search must be done
    :param num: the number of sentences to retrieve

    :return: the list of sentences containing the verb
    """
    i = 0
    sentences = np.array([])
    while len(sentences) < num:
        review = corpus.raw(corpus.fileids()[i])
        if verb in review.split():
            for s in sent_tokenize(review): # le recensioni contengono più frasi, le dividiamo considerandole singolarmente
                if verb in s.split():
                    sentences = np.append(sentences, s)
        i += 1
    return sentences


sentences = get_sentences(VERB, movie_reviews, 1000)
print(len(sentences))
sentences[:5]

1001


array(['they get into an accident .',
       "now i personally don't mind trying to unravel a film every now and then , but when all it does is give me the same clue over and over again , i get kind of fed up after a while , which is this film's biggest problem .",
       'okay , we get it .',
       "you're more likely to get a kick out of her work in halloween h20 .",
       'all this to get to the familiar yet offensive " revelation " that sexual deviants are not , indeed , monsters but everyday people like you and me .'],
      dtype='<U887')

Effettuiamo __parsing__ e __disambiguazione__.

Partiamo dal __parsing__, il cui fine è quello di estrarre i _fillers_ del verbo dalle frasi.
Essendo il verbo scelto un verbo transitivo, consideriamo l'arità pari a 2 (soggetto ed oggetto), otteniamo così delle triple _(subj, verb, obj)_.

In [21]:
nlp = spacy.load("en_core_web_sm") # usiamo la libreria SpaCy per ottenere le dipendenze delle frasi
fillers = [] # inseriremo qui il soggetto e l'oggetto del verbo
index = [] # teniamo traccia dell'indice della frase per poterla recuperare nelle operazioni successive (WSD)
for i in range(len(sentences)):
    doc = nlp(str(sentences[i]))
    df = pd.DataFrame(data=[(token.dep_, token.head, [c for c in token.children]) for token in doc], index=[token.text for token in doc], columns=['token.dep_', 'token.head', 'token.children']) # indicizziamo tutto in un dataframe per recuperare i dati più velocemente
    childrens = df.loc[VERB, 'token.children'] # prendiamo i termini che sono in dipendenza del verbo indicato
    subj = None
    obj = None
    for c in childrens:
        try:
            c_dep = df.loc[str(c), 'token.dep_'] # prendiamo il valore della dipendenza
            try:
                c_dep = c_dep.iloc[0] # nel caso in cui la stessa parola compaia più volte nel dataframe prendiamo la prima occorrenza
            except AttributeError:
                pass
            # salviamo solo soggetto ed oggetto se presenti
            if str(c_dep) == 'nsubj':
                subj = str(c)
            if str(c_dep) == 'dobj':
                obj = str(c)
        except KeyError:
            # se ci sono più possibili triple nella stessa frase iteriamo su ognuna di esse
            for sub_child in c:
                c_dep = df.loc[str(sub_child), 'token.dep_'] # prendiamo il valore della dipendenza
                try:
                    c_dep = c_dep.iloc[0] # nel caso in cui la stessa parola compaia più volte nel dataframe ne prendiamo la prima occorrenza
                except AttributeError:
                    pass
                # salviamo solo soggetto ed oggetto se presenti
                if str(c_dep) == 'nsubj':
                    subj = str(sub_child)
                if str(c_dep) == 'dobj':
                    obj = str(sub_child)
            if subj and obj is not None:
                fillers.append((subj, VERB, obj))
                index.append(i)
                # azzerando i valori di subj ed obj possiamo catturare anche le altre triple della frase, altrimenti ci sarebbero due triple uguali
                subj = None
                obj = None
    if subj and obj is not None:
        fillers.append((subj, VERB, obj))
        index.append(i)

print('Numero di triple estratte:', len(fillers)) # le triple complete (senza nessun campo None) sono 266
fillers[:5]

Numero di triple estratte: 266


[('we', 'get', 'it'),
 ('that', 'get', 'laughs'),
 ('who', 'get', 'kicks'),
 ('we', 'get', 'money'),
 ('movie', 'get', 'star')]

In [22]:
# indicizziamo tutto in un dataframe per mostrare i risultati in maniera più organizzata
df = pd.DataFrame(data=fillers, index=index, columns=['subj','verb', 'obj'])
df[:5]

,subj,verb,obj
2,we,get,it
5,that,get,laughs
6,who,get,kicks
8,we,get,money
9,movie,get,star


Procediamo ora con la __disambiguazione__ dei fillers, in questo modo avremo il synset di _WordNet_ che ne rappresenta il senso. L'algoritmo di WSD utilizato è quello di _Lesk_, che ritorna il synset più adatto al termine oppure _None_ nel caso in cui non riesca ad ottenerne uno.

In [23]:
syn_subj = []
syn_obj = []
for i, subj, obj in zip(df.index, df['subj'], df['obj']):
    sent = sentences[i].split()
    syn_subj.append(lesk(sent, subj))
    syn_obj.append(lesk(sent, obj))
df['subj syn'] = syn_subj
df['obj syn'] = syn_obj
df[:5]

,subj,verb,obj,subj syn,obj syn
2,we,get,it,None,Synset('information_technology.n.01')
5,that,get,laughs,None,Synset('joke.n.01')
6,who,get,kicks,Synset('world_health_organization.n.01'),Synset('recoil.n.01')
8,we,get,money,None,Synset('money.n.03')
9,movie,get,star,Synset('movie.n.01'),Synset('star_topology.n.01')


Notiamo che in molte delle triple Lesk non riesce a trovare il senso del soggetto, mentre non ha grosse difficoltà a trovare quello dell'oggetto. Per avere un'idea contiamo il numero di elementi per cui è stato trovato il synset.

In [24]:
print('Numero di elementi non nulli fra i subj:  ', df['subj syn'].notnull().sum())
print('Numero di elementi non nulli fra gli obj: ', df['obj syn'].notnull().sum())
print('Lunghezza totale delle colonne:  ', len(df['subj syn']))

Numero di elementi non nulli fra i subj:   89
Numero di elementi non nulli fra gli obj:  234
Lunghezza totale delle colonne:   266


## Recupero dei semantic types

Passiamo adesso alla fase di estrazione dei __semantic types__. Li otteniamo accedendo a WordNet usando la funzione _lexname()_ presente in NLTK, che ritorna il super senso del termine.

In [25]:
df['super sense subj'] = df['subj syn'].apply(lambda x: x.lexname() if x is not None else None)
df['super sense obj'] = df['obj syn'].apply(lambda x: x.lexname() if x is not None else None)
df[:5]

,subj,verb,obj,subj syn,obj syn,super sense subj,super sense obj
2,we,get,it,None,Synset('information_technology.n.01'),None,noun.cognition
5,that,get,laughs,None,Synset('joke.n.01'),None,noun.communication
6,who,get,kicks,Synset('world_health_organization.n.01'),Synset('recoil.n.01'),noun.group,noun.event
8,we,get,money,None,Synset('money.n.03'),None,noun.possession
9,movie,get,star,Synset('movie.n.01'),Synset('star_topology.n.01'),noun.communication,noun.cognition


## Clusterizzazione dei semantic types

Visualizziamo in maniera aggregata quali sono le categorie che compaiono con più frequenza fra i _semantic types_.

Prendiamo in esame la colonna dei _soggetti_.

In [26]:
print(len(df['super sense subj'].value_counts()))
df['super sense subj'].value_counts()

20


noun.substance        25
noun.person           16
noun.group             6
adj.all                6
noun.communication     6
noun.cognition         5
noun.artifact          4
noun.quantity          4
noun.state             2
verb.change            2
noun.event             2
verb.motion            2
noun.Tops              2
verb.competition       1
noun.food              1
noun.act               1
noun.time              1
noun.possession        1
verb.consumption       1
noun.location          1
Name: super sense subj, dtype: int64

Ora consideriamo quella degli _oggetti_.

In [27]:
print(len(df['super sense obj'].value_counts()))
df['super sense obj'].value_counts()

34


noun.communication    32
noun.cognition        25
noun.act              18
noun.person           16
noun.attribute        15
noun.state            14
noun.possession        9
noun.quantity          9
noun.artifact          9
verb.communication     8
adj.all                8
verb.possession        7
verb.social            6
verb.cognition         6
verb.perception        5
noun.location          5
noun.group             5
verb.change            4
verb.motion            4
noun.Tops              3
noun.time              3
noun.event             3
noun.phenomenon        2
verb.competition       2
noun.feeling           2
noun.substance         2
adv.all                2
verb.body              2
verb.consumption       2
verb.creation          2
verb.contact           1
noun.object            1
noun.animal            1
verb.stative           1
Name: super sense obj, dtype: int64

Osservando le due distribuzioni si nota subito come la varianza di termini utilizzati sia molto maggiore per gli oggetti (34 categorie differenti).
I soggetti invece sono in minor numero e concentrati nelle categorie _noun.substance_ e _noun.person_.

Produciamo ora in output le frequenze delle triple _soggetto-verbo-oggetto_.

In [28]:
df.groupby(['super sense subj', 'verb', 'super sense obj']).size().sort_values(ascending=False)

super sense subj    verb  super sense obj   
noun.group          get   noun.act              3
noun.substance      get   noun.possession       3
noun.person         get   noun.communication    2
noun.substance      get   noun.state            2
noun.person         get   verb.possession       2
                          noun.attribute        2
                          noun.artifact         2
                          noun.act              2
noun.quantity       get   noun.feeling          2
noun.group          get   noun.attribute        2
noun.substance      get   noun.cognition        2
noun.communication  get   noun.cognition        2
noun.substance      get   noun.person           2
                          verb.social           2
noun.person         get   noun.state            2
noun.substance      get   verb.cognition        1
noun.person         get   verb.perception       1
verb.motion         get   verb.possession       1
noun.possession     get   noun.time             1
verb.

Dalle frequenze delle triple notiamo risultati piuttosto bassi, si possono comunque individuare alcuni significati per il verbo selezionato (_get_), come quello di effettuare un'azione (_noun.group, get, noun.act_) e quello di possesso (_noun.substance, get, noun.possession_).